In [26]:
class myClass():
    
    noCandles = 2000
    
    pars = {'loadMethod' : None,
            'dbName' : None,
            'fileName' : None,
            'loadFuntion': None}
    
    @classmethod
    def reset(cls):
        """Reset class parameters"""
        for par in cls.pars:
            cls.pars[par] = None

    @classmethod
    def init(cls, string):
        
        cls.reset()
        
        if string['loadMethod'] == 'influxdb':
            cls.pars['loadMethod'] = string['loadMethod']
            cls.pars['dbName'] = string['dbName']
            cls.pars['loadFunction'] = string['loadFunction']
            
        elif string['loadMethod'] == 'file':
            cls.pars['loadMethod'] = string['loadMethod']
            cls.pars['fileName'] = string['fileName']
            cls.pars['loadFunction'] = string['loadFunction']
            
        else:
            raise ValueError

In [29]:
def fun(a, b, client):
    print('execute fun')
    print(client)
    
myClass.init({'loadMethod': 'influxdb', 'dbName': 'name1', 'loadFunction': fun})

In [30]:
myClass.pars
keyargs = {'client': 3}
myClass.pars['loadFunction'](1,2, **keyargs)

execute fun
3


In [4]:
myClass.init({'loadMethod': 'file', 'fileName': 'name2'})

In [5]:
myClass.pars

{'loadMethod': 'file', 'dbName': None, 'fileName': 'name2'}

In [7]:
import json
json.dumps('some text goes here')

'"some text goes here"'

In [32]:
def fun1():
    print('executing fun1')

def fun2():
    return fun1()

fun2()

executing fun1


In [57]:
from influxdb import DataFrameClient
def loadDataFromInfluxdb(dtLimit, direction, client, noCandles):
    """Actual implementation of data loading from Influxdb"""
    if direction == 'left':
        query = 'SELECT "time", "open", "high", "low", "close" FROM "rates" WHERE time < \'%s\' AND ("status" = \'C\' OR "status" = \'A\') ORDER BY DESC LIMIT %i' % (dtLimit, noCandles)
    elif direction == 'right':
        query = 'SELECT "time", "open", "high", "low", "close" FROM "rates" WHERE time > \'%s\' AND ("status" = \'C\' OR "status" = \'A\') ORDER BY ASC LIMIT %i' % (dtLimit, noCandles)
    try:
        rates = client.query(query)['rates']
    except KeyError:
        raise Exception('No such data exist in database!')
    return rates

In [55]:
client = DataFrameClient(host='127.0.0.1', port=8086, database='oanda.eurusd.M1')
rates = loadDataFromInfluxdb('2011-01-03T13:42:00Z', 'left', client, 2000)

In [56]:
print(rates)

                              open     high      low    close
2011-01-03 00:00:00+00:00  1.33417  1.33451  1.33405  1.33423
2011-01-03 00:01:00+00:00  1.33425  1.33464  1.33425  1.33459
2011-01-03 00:02:00+00:00  1.33457  1.33460  1.33428  1.33432
2011-01-03 00:03:00+00:00  1.33434  1.33453  1.33401  1.33401
2011-01-03 00:04:00+00:00  1.33398  1.33398  1.33377  1.33387
...                            ...      ...      ...      ...
2011-01-03 13:37:00+00:00  1.33454  1.33457  1.33438  1.33438
2011-01-03 13:38:00+00:00  1.33435  1.33442  1.33415  1.33442
2011-01-03 13:39:00+00:00  1.33444  1.33464  1.33442  1.33464
2011-01-03 13:40:00+00:00  1.33462  1.33506  1.33462  1.33503
2011-01-03 13:41:00+00:00  1.33505  1.33536  1.33487  1.33494

[816 rows x 4 columns]
